In [1]:
import sys
sys.path.append('./MatchZoo-py/')

In [13]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [14]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [15]:
preprocessor = mz.models.DRMM.get_default_preprocessor()

In [16]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6510.95it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3639.01it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2108239.88it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8030.69it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4104.93it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 771983.68it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7863.31it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [17]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fded4252908>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fded4252ac8>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [18]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [19]:
histgram_callback = mz.dataloader.callbacks.Histogram(
    embedding_matrix, bin_size=30, hist_mode='LCH'
)

trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10,
    callbacks=[histgram_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    callbacks=[histgram_callback]
)

In [20]:
padding_callback = mz.models.DRMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    device='cpu',
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [21]:
model = mz.models.DRMM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding'] = embedding_matrix
model.params['hist_bin_size'] = 30
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMM(
  (embedding): Embedding(30058, 300)
  (attention): Attention(
    (linear): Linear(in_features=300, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=30, out_features=10, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Tanh()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  9018023


In [22]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    device='cpu',
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

[Iter-255 Loss-2.300]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4853 - normalized_discounted_cumulative_gain@5(0.0): 0.5517 - mean_average_precision(0.0): 0.518



[Iter-510 Loss-2.143]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.544 - normalized_discounted_cumulative_gain@5(0.0): 0.6008 - mean_average_precision(0.0): 0.5671



[Iter-765 Loss-2.085]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5649 - normalized_discounted_cumulative_gain@5(0.0): 0.6066 - mean_average_precision(0.0): 0.5747



[Iter-1020 Loss-2.033]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5496 - normalized_discounted_cumulative_gain@5(0.0): 0.5959 - mean_average_precision(0.0): 0.5588



[Iter-1275 Loss-1.984]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.567 - normalized_discounted_cumulative_gain@5(0.0): 0.6085 - mean_average_precision(0.0): 0.5738



[Iter-1530 Loss-1.944]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.56 - normalized_discounted_cumulative_gain@5(0.0): 0.609 - mean_average_precision(0.0): 0.5723



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

